In [1]:
import river
import river.datasets as datasets
from river import stream
from pprint import pprint
import os
from scipy.io import arff
import pandas as pd
from river import tree
from river import evaluate
from river import metrics

import numpy as np
from  streams.stream_section import StreamSection
from streams.utils import FL, FU, generate_stream_section
from my_datasets.getting_datasets import get_LED,get_Hyperplane,get_Electricity,get_AGRAWL,get_Airlines,get_CoverType,get_RandomRBF
from train_and_eval import train_and_evaluate
from functools import partial
from train_and_eval import train_and_evaluate
from river.metrics.accuracy import Accuracy
from multiprocessing.pool import ThreadPool
from river.metrics import CohenKappa
from river.datasets.synth import ConceptDriftStream
from tqdm import tqdm

# methods
from river.dummy import NoChangeClassifier,PriorClassifier
from river.cluster import CluStream
from river.forest.adaptive_random_forest import ARFClassifier
from river.tree.hoeffding_adaptive_tree_classifier import  HoeffdingAdaptiveTreeClassifier
from river.naive_bayes import GaussianNB
from semisupervised_methods.clustream_and_label import CluserAndLabel
from semisupervised_methods.incremental_classifier import IncrementalClassifer
from semisupervised_methods.one_nearest_neighbour import oneNNClassifer
# from semisupervised_methods.my_method import MyClassifier
from river.drift import ADWIN, PageHinkley
import operator
from semisupervised_methods.ICLC import ICLC
from river.cluster import KMeans


In [2]:
import logging
import datetime
now = datetime.datetime.now()
date_time = now.strftime("%d_%m_%Y_%H_%M_%S")
logging.basicConfig(filename=f'logs\\{date_time}.log', filemode='w',
                    format='%(asctime)s - %(message)s', level=logging.INFO, datefmt='%d-%b-%y %H:%M:%S')
logger = logging.getLogger()

In [3]:
led3 = get_LED(0.1,True,3)
led5 = get_LED(0.1,True,5)
led7 = get_LED(0.1,True,7)
st = [(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led3.take(3750))]
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led5.take(7500))])
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led7.take(8750))])
initial_stream = StreamSection('LED_abrupt', st, True)
Q=[(0,20000)]
probas = [0.1,0.2]  # probabilities of dropping the label
HT1 = HoeffdingAdaptiveTreeClassifier
ARF1 = ARFClassifier
NB1= GaussianNB
incClasif2= IncrementalClassifer
incClasif1= IncrementalClassifer
incClasif3= IncrementalClassifer
m_params = [
    {'threshold':0.7,
  'classifier':NB1,
  'params':{}}
]
delay_type =1
methods = [incClasif1,incClasif2]
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['NB07'],
                         metric_fun=CohenKappa, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger)


IndexError: list index out of range

In [ ]:
led3 = get_LED(0.1,True,3)
led5 = get_LED(0.1,True,5)
led7 = get_LED(0.1,True,7)
st = [(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led3.take(3750))]
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led5.take(7500))])
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led7.take(8750))])
initial_stream = StreamSection('LED_abrupt', st, True)
Q=[(0,20000)]
probas = [0.1,0.2]  # probabilities of dropping the label
dummy1 = PriorClassifier
dummy2 = NoChangeClassifier
incClasif2= IncrementalClassifer
incClasif1= IncrementalClassifer
m_params = [

  {'threshold':0.7,
  'classifier':dummy1,
  'params':{}},
  {'threshold':0.7,
  'classifier':dummy2,
  'params':{}}
]
delay_type =1
methods = [incClasif1,incClasif2]
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['Prior','Majority'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger)


In [4]:
led3 = get_LED(0.1,True,3)
led5 = get_LED(0.1,True,5)
led7 = get_LED(0.1,True,7)
st = [(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led3.take(3750))]
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led5.take(7500))])
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led7.take(8750))])
initial_stream = StreamSection('LED_abrubt', st, True)
Q=[(0,20000)]
probas = [0.1,0.2]  # probabilities of dropping the label
my_method = ICLC
Cl1= KMeans
ARF = ARFClassifier
m_params = [
 {  'classifer':ARF,
  'clustering_method':Cl1,
  'clustering_params':{},
  'classifier_params':{},
  'drift_detector': ADWIN,
  'nu':100}
]
methods = [my_method]
delay_type = 1 # constant delay

# # run the experiment 
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['ICLC'],
                         metric_fun=CohenKappa, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger)


LED_abrubt_0_20000
LED_abrubt_0_20000_constant_delay
LED_abrubt_0_20000_constant_delay_ssl_0.1_0_20000
LED_abrubt_0_20000_constant_delay_ssl_0.2_0_20000


In [5]:
led3 = get_LED(0.1,True,3)
led5 = get_LED(0.1,True,5)
led7 = get_LED(0.1,True,7)
st = [(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led3.take(3750))]
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led5.take(7500))])
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led7.take(8750))])
initial_stream = StreamSection('LED_abrupt', st, True)
Q=[(0,20000)]
probas = [0.1,0.2]  # probabilities of dropping the label
clustream = CluserAndLabel

m_params = [
    {'seed':123}]
methods = [clustream]
delay_type = 1 # constant delay

# # run the experiment 
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['CluStream'],
                         metric_fun=CohenKappa, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger)


LED_abrubt_0_20000
LED_abrubt_0_20000_constant_delay
LED_abrubt_0_20000_constant_delay_ssl_0.1_0_20000
LED_abrubt_0_20000_constant_delay_ssl_0.2_0_20000


In [ ]:
stream = get_Hyperplane(n_features=10, n_drift_features=10, mag_change= 0.001,
                   noise_percentage = 0.01)
initial_stream = generate_stream_section(stream,'HyperPlane','synth',start=0,stop=10000)
Q=[(0,len(initial_stream.stream))]
HT1 = HoeffdingAdaptiveTreeClassifier
ARF1 = ARFClassifier
NB1= GaussianNB
incClasif2= IncrementalClassifer
incClasif1= IncrementalClassifer
incClasif3= IncrementalClassifer
m_params = [

  {'threshold':0.7,
  'classifier':ARF1,
  'params':{'seed':123}},
  {'threshold':0.7,
  'classifier':HT1,
  'params':{'seed':123}},
    {'threshold':0.7,
  'classifier':NB1,
  'params':{}}
]
delay_type =1
methods = [incClasif1,incClasif2,incClasif3]
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['ARF07','HT07','NB07'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=500,logger = logger)

In [4]:
# get the data stream
led3 = get_LED(0.1,True,3)
led5 = get_LED(0.1,True,5)
led7 = get_LED(0.1,True,7)
st = [(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led3.take(3750))]
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led5.take(7500))])
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led7.take(8750))])
initial_stream = StreamSection('LED_abrubt', st, True)
Q = [(0, 20000)]  # stream secion boundries
probas = [0.1,0.2]  # probabilities of dropping the label

clustream = CluserAndLabel

m_params = [
    {'seed':123}]
methods = [clustream]
delay_type = 1 # constant delay

# # run the experiment 
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['CluStream'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=300,logger = logger)


In [5]:
import logging
import datetime
now = datetime.datetime.now()
date_time = now.strftime("%d_%m_%Y_%H_%M_%S")
logging.basicConfig(filename=f'logs\\{date_time}.log', filemode='w',
                    format='%(asctime)s - %(message)s', level=logging.INFO, datefmt='%d-%b-%y %H:%M:%S')
logger = logging.getLogger()

In [6]:
led3 = get_LED(0.1,True,3)
led5 = get_LED(0.1,True,5)
led7 = get_LED(0.1,True,7)
st = [(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led3.take(3750))]
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led5.take(7500))])
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led7.take(8750))])
initial_stream = StreamSection('LED_abrubt', st, True)
Q = [(0, 20000)]  # stream secion boundries
probas = [0.1,0.2]  # probabilities of dropping the label

clustream = CluserAndLabel

m_params = [
    {'seed':123}]
methods = [clustream]
delay_type = 1 # constant delay

# # run the experiment 
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['CluStream'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=300,logger = logger)

In [11]:
dataset_name = 'Cover_Type'
Q=[(0,20000)]
probas = [0.1,0.2,0.5]
HT1 = HoeffdingAdaptiveTreeClassifier
ARF1 = ARFClassifier
NB1= GaussianNB
incClasif2= IncrementalClassifer
incClasif1= IncrementalClassifer
incClasif3= IncrementalClassifer
m_params = [

  {'threshold':0.9,
  'classifier':ARF1,
  'params':{'seed':123}},
  {'threshold':0.9,
  'classifier':HT1,
  'params':{'seed':123}},
    {'threshold':0.9,
  'classifier':NB1,
  'params':{}}
]
methods = [incClasif1,incClasif2,incClasif3]
res = train_and_evaluate(initial,dataset_name,Q,probas,methods,m_params,['ARF09','HT09','NB09'],Accuracy,delay_type = 1,B=50,K=10,delay=1000)